In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import yfinance as yf
import uuid


In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
keyspace_name='settrade_api'
session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)

In [4]:
session.set_keyspace('settrade_api')

In [5]:
session.execute("""
CREATE TABLE IF NOT EXISTS financial_data (
    ticker TEXT PRIMARY KEY,
    name TEXT,
    sector TEXT,
    industry TEXT,
    total_debt DOUBLE,
    total_assets DOUBLE,
    total_revenue DOUBLE,
    interest_income DOUBLE,
    long_business_summary TEXT
)
""")

In [6]:

# ✅ เตรียม Insert Statement นอกลูป
insert_stmt = session.prepare("""
INSERT INTO financial_data (
    ticker, name, sector, industry,
    total_debt, total_assets, total_revenue,
    interest_income, long_business_summary
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

# ✅ โหลดรายชื่อหุ้น
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().astype(str).tolist()
    print(f"✅ Loaded {len(symbols)} symbols from Excel.")
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# 🔧 แปลงเป็น ticker แบบที่ใช้กับ yfinance
tickers = [symbol.strip().upper() + ".BK" for symbol in symbols]

# 🔁 ดึงและโหลดข้อมูลทีละตัว
for ticker in tickers:
    try:
        print(f"📥 Fetching {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info

        # 🔍 ดึงข้อมูลที่จำเป็น
        name = info.get('shortName', '')
        sector = info.get('sector', '')
        industry = info.get('industry', '')
        summary = info.get('longBusinessSummary', '')

        total_debt = float(info.get('totalDebt', 0.0) or 0.0)
        total_assets = float(info.get('totalAssets', 1.0) or 1.0)
        total_revenue = float(info.get('totalRevenue', 0.0) or 0.0)

        # 🔸 ประเมิน interest income จาก EBITDA margin
        ebitda_margin = float(info.get('ebitdaMargins', 0.0) or 0.0)
        interest_income = ebitda_margin * total_assets

        # ✅ Insert ลง Cassandra
        session.execute(insert_stmt, (
            ticker, name, sector, industry,
            total_debt, total_assets, total_revenue,
            interest_income, summary
        ))

        print(f"✅ Inserted {ticker} successfully.")

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")


❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง
📥 Fetching PTT.BK...
✅ Inserted PTT.BK successfully.
📥 Fetching AOT.BK...
✅ Inserted AOT.BK successfully.
📥 Fetching SCB.BK...
✅ Inserted SCB.BK successfully.
📥 Fetching CPALL.BK...
✅ Inserted CPALL.BK successfully.
📥 Fetching ADVANC.BK...
✅ Inserted ADVANC.BK successfully.


In [7]:
# 🔁 ดึงและโหลดทีละตัว
for ticker in tickers:
    try:
        print(f"📥 Fetching {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info

        # 🔍 ดึงข้อมูลที่จำเป็น
        name = info.get('shortName', '')
        sector = info.get('sector', '')
        industry = info.get('industry', '')
        summary = info.get('longBusinessSummary', '')

        total_debt = float(info.get('totalDebt', 0.0) or 0.0)
        total_assets = float(info.get('totalAssets', 1.0) or 1.0)
        total_revenue = float(info.get('totalRevenue', 0.0) or 0.0)

        # interest income ไม่มีใน yfinance → ประเมินจาก EBITDA Margin
        ebitda_margin = float(info.get('ebitdaMargins', 0.0) or 0.0)
        interest_income = ebitda_margin * total_assets
        insert_stmt = session.prepare("""
INSERT INTO financial_data (
    ticker, name, sector, industry,
    total_debt, total_assets, total_revenue,
    interest_income, long_business_summary
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")
        # ✅ Insert ลง Cassandra
        session.execute(insert_stmt, (
            ticker, name, sector, industry,
            total_debt, total_assets, total_revenue,
            interest_income, summary
        ))

        print(f"✅ {ticker} inserted.")

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

📥 Fetching PTT.BK...
✅ PTT.BK inserted.
📥 Fetching AOT.BK...
✅ AOT.BK inserted.
📥 Fetching SCB.BK...
✅ SCB.BK inserted.
📥 Fetching CPALL.BK...
✅ CPALL.BK inserted.
📥 Fetching ADVANC.BK...
✅ ADVANC.BK inserted.
❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง
